<a href="https://colab.research.google.com/github/PhuongDang1/SIT-Research-Project/blob/main/Current_Research_Project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Relevant Libraries**

In [2]:
%tensorflow_version 2.x
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, GRU,Reshape, Dense
from numpy import array
import tensorflow as tf
from datetime import datetime
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading the data into a variable**

In [3]:
dataset = pd.read_excel("/content/drive/My Drive/Covid_Dataset.xlsx")
dataset

,dateRep,cases,deaths,countriesAndTerritories,Population Density 2020 (P/KM2),continentExp,Wind Speed Minimum,Wind Speed Maximum,Wind Speed Median,Wind Speed Variance
0,2019-12-31,0,0,India,464,Asia,0.4,2.2,1.7,3.58
1,2020-01-01,0,0,India,464,Asia,0.5,2.6,0.8,4.89
2,2020-01-02,0,0,India,464,Asia,0.5,1.6,0.7,1.40
3,2020-01-03,0,0,India,464,Asia,0.1,9.9,0.9,20.86
4,2020-01-04,0,0,India,464,Asia,0.1,11.8,0.8,20.96
...,...,...,...,...,...,...,...,...,...,...
1387,2020-12-10,220025,3124,United_States_of_America,36,America,0.1,18.0,3.8,114.01
1388,2020-12-11,224680,2748,United_States_of_America,36,America,0.1,12.0,2.5,51.39
1389,2020-12-12,234633,3343,United_States_of_America,36,America,0.1,8.5,2.5,25.50
1390,2020-12-13,216017,2315,United_States_of_America,36,America,0.8,15.0,3.6,77.38


# **Preprocessing for confirmed cases test only**

In [4]:
new_index = dataset.iloc[:,0]
new_index
values = dataset.set_index(new_index)
values = values.iloc[:,1:]
values = values.iloc[:1500,0:2].astype('float32')
values
labels = dataset.iloc[:1500,-9]
labels

from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(values, labels, test_size=0.20, random_state=88)

In [5]:
train_labels = pd.array(train_labels)
test_labels = pd.array(test_labels)

In [6]:
train = train_data
scalers={}
for i in train_data.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s= np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_data
for i in test_data.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s= np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [7]:
def split_series(series, n_past, n_future):
  # n_past ==> no of past observations
  # n_future ==> no of future observations 
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [8]:
n_past = 10
n_future = 20
n_features = 2

In [9]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

# **Designing the Model of LSTM**

In [ ]:
#LSTM
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(2, activation='tanh',return_state=True) 
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
decoder_l1 = tf.keras.layers.LSTM(2, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
model_LSTM = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

model_LSTM.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 2)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 2), (None, 2 40          input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 20, 2)        0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 20, 2)        40          repeat_vector[0][0]              
                                                                 lstm[0][1]                   

In [ ]:
from keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

# **Training of LSTM and evaluation**

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

#loss test
model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy', 'mean_squared_error'])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy',mean_squared_error])

#optimizer test
#model_LSTM.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])

history_LSTM = model_LSTM.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=25,verbose=1,callbacks=[reduce_lr])

Epoch 1/100
44/44 [==============================] - 5s 35ms/step - loss: 0.7191 - accuracy: 0.4793 - mean_squared_error: 0.7191 - val_loss: 0.6526 - val_accuracy: 0.5516 - val_mean_squared_error: 0.6526
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 0.5744 - accuracy: 0.5412 - mean_squared_error: 0.5744 - val_loss: 0.4900 - val_accuracy: 0.5868 - val_mean_squared_error: 0.4900
Epoch 3/100
44/44 [==============================] - 1s 15ms/step - loss: 0.4197 - accuracy: 0.5432 - mean_squared_error: 0.4197 - val_loss: 0.3557 - val_accuracy: 0.5862 - val_mean_squared_error: 0.3557
Epoch 4/100
44/44 [==============================] - 1s 14ms/step - loss: 0.3161 - accuracy: 0.5435 - mean_squared_error: 0.3161 - val_loss: 0.2747 - val_accuracy: 0.5862 - val_mean_squared_error: 0.2747
Epoch 5/100
44/44 [==============================] - 1s 14ms/step - loss: 0.2488 - accuracy: 0.5429 - mean_squared_error: 0.2488 - val_loss: 0.2170 - val_accuracy: 0.5868 - val_mean_sq

In [ ]:
from sklearn.metrics import mean_squared_error
loss, accuracy,mean_squared_error = model_LSTM.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % accuracy)
print('Error Rate: %f' % (1 - accuracy))
print('Mean Squared Error: %f' % mean_squared_error)

Accuracy: 0.582200
Error Rate: 0.417800
Mean Squared Error: 0.060058


# **Training of GRU and evaluation**

In [10]:
# GRU
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.GRU(2,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

encoder_l2 = tf.keras.layers.GRU(2, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 = tf.keras.layers.GRU(2, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.GRU(2, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)

model_GRU = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_GRU.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 2)]      0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 10, 2), (Non 36          input_1[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, 2), (None, 2 36          gru[0][0]                        
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 20, 2)        0           gru_1[0][0]                      
______________________________________________________________________________________________

In [15]:
#loss test
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy',mean_squared_error])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy',mean_squared_error])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy',mean_squared_error])

#optimizer test
#model_GRU.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compilee(optimizer=tf.keras.optimizers.Adadelta(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])

In [16]:
model_GRU.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy','mean_squared_error'])
history_GRU = model_GRU.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=25,verbose=1,callbacks=[reduce_lr])

Epoch 1/100
44/44 [==============================] - 9s 52ms/step - loss: 0.6600 - accuracy: 0.5351 - mean_squared_error: 0.6600 - val_loss: 0.4165 - val_accuracy: 0.5666 - val_mean_squared_error: 0.4165
Epoch 2/100
44/44 [==============================] - 1s 26ms/step - loss: 0.2744 - accuracy: 0.5349 - mean_squared_error: 0.2744 - val_loss: 0.1541 - val_accuracy: 0.5666 - val_mean_squared_error: 0.1541
Epoch 3/100
44/44 [==============================] - 1s 25ms/step - loss: 0.1186 - accuracy: 0.5349 - mean_squared_error: 0.1186 - val_loss: 0.0832 - val_accuracy: 0.5666 - val_mean_squared_error: 0.0832
Epoch 4/100
44/44 [==============================] - 1s 24ms/step - loss: 0.0863 - accuracy: 0.5350 - mean_squared_error: 0.0863 - val_loss: 0.0704 - val_accuracy: 0.5666 - val_mean_squared_error: 0.0704
Epoch 5/100
44/44 [==============================] - 1s 24ms/step - loss: 0.0793 - accuracy: 0.5350 - mean_squared_error: 0.0793 - val_loss: 0.0657 - val_accuracy: 0.5666 - val_mean_sq

In [17]:
from sklearn.metrics import mean_squared_error
loss, accuracy,mean_squared_error = model_GRU.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % accuracy)
print('Error Rate: %f' % (1 - accuracy))
print('Mean Squared Error: %f' % mean_squared_error)

Accuracy: 0.556800
Error Rate: 0.443200
Mean Squared Error: 0.059493


# **Preprocessing for confirmed cases with Wind Speed and Population density factors**

In [18]:
new_index = dataset.iloc[:,0]
new_index
values = dataset.set_index(new_index)
values = values.iloc[:,1:]
values = values.iloc[:1500,[0,1,3,5,6,7,8]].astype('float32')
values
labels = dataset.iloc[:1500,-9]
labels

from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(values, labels, test_size=0.20, random_state=88)

In [19]:
train_labels = pd.array(train_labels)
test_labels = pd.array(test_labels)

In [20]:
train = train_data
scalers={}
for i in train_data.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s= np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_data
for i in test_data.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s= np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [21]:
def split_series(series, n_past, n_future):
  # n_past ==> no of past observations
  # n_future ==> no of future observations 
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [22]:
n_past = 10
n_future = 20
n_features = 7

In [23]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

# **Designing the Model of LSTM**

In [28]:
#LSTM
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(2, activation='tanh',return_state=True) 
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
decoder_l1 = tf.keras.layers.LSTM(2, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
model_LSTM2 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

model_LSTM2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 7)]      0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 2), (None, 2 80          input_3[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 20, 2)        0           lstm_2[0][0]                     
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 20, 2)        40          repeat_vector_2[0][0]            
                                                                 lstm_2[0][1]               

In [30]:
from keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

# **Training the Model of LSTM using factors and evaluation**

In [31]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

#loss test
model_LSTM2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy', 'mean_squared_error'])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy',mean_squared_error])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy',mean_squared_error])

#optimizer test
#model_LSTM.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_LSTM.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_LSTM.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])

history_LSTM2 = model_LSTM2.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=25,verbose=1,callbacks=[reduce_lr])

Epoch 1/100
44/44 [==============================] - 4s 28ms/step - loss: 0.6516 - accuracy: 0.0456 - mean_squared_error: 0.6516 - val_loss: 0.6101 - val_accuracy: 0.0338 - val_mean_squared_error: 0.6101
Epoch 2/100
44/44 [==============================] - 1s 12ms/step - loss: 0.5624 - accuracy: 0.0262 - mean_squared_error: 0.5624 - val_loss: 0.5248 - val_accuracy: 0.0294 - val_mean_squared_error: 0.5248
Epoch 3/100
44/44 [==============================] - 1s 12ms/step - loss: 0.4798 - accuracy: 0.0251 - mean_squared_error: 0.4798 - val_loss: 0.4458 - val_accuracy: 0.0336 - val_mean_squared_error: 0.4458
Epoch 4/100
44/44 [==============================] - 1s 12ms/step - loss: 0.4068 - accuracy: 0.0306 - mean_squared_error: 0.4068 - val_loss: 0.3806 - val_accuracy: 0.0348 - val_mean_squared_error: 0.3806
Epoch 5/100
44/44 [==============================] - 1s 12ms/step - loss: 0.3493 - accuracy: 0.0418 - mean_squared_error: 0.3492 - val_loss: 0.3314 - val_accuracy: 0.0594 - val_mean_sq

In [32]:
from sklearn.metrics import mean_squared_error
loss, accuracy,mean_squared_error = model_LSTM2.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % accuracy)
print('Error Rate: %f' % (1 - accuracy))
print('Mean Squared Error: %f' % mean_squared_error)

Accuracy: 0.693400
Error Rate: 0.306600
Mean Squared Error: 0.163875


# **Designing of GRU model using factors**

In [40]:
# GRU
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.GRU(2,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

encoder_l2 = tf.keras.layers.GRU(2, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 = tf.keras.layers.GRU(2, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.GRU(2, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)

model_GRU2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_GRU2.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 10, 7)]      0                                            
__________________________________________________________________________________________________
gru_12 (GRU)                    [(None, 10, 2), (Non 66          input_6[0][0]                    
__________________________________________________________________________________________________
gru_13 (GRU)                    [(None, 2), (None, 2 36          gru_12[0][0]                     
__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 20, 2)        0           gru_13[0][0]                     
____________________________________________________________________________________________

In [41]:
#loss test
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_GRU2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy',mean_squared_error])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy',mean_squared_error])
#model_GRU.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.1e-3), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy',mean_squared_error])

#optimizer test
#model_GRU.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy', 'mean_squared_error'])
#model_GRU.compilee(optimizer=tf.keras.optimizers.Adadelta(learning_rate=.1e-3), loss=tf.keras.losses.Huber(),metrics=['accuracy',mean_squared_error])

# **Training the Model of LSTM using factors and evaluation**

In [42]:
model_GRU2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy','mean_squared_error'])
history_GRU = model_GRU2.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=25,verbose=1,callbacks=[reduce_lr])

Epoch 1/100
44/44 [==============================] - 8s 50ms/step - loss: 0.6568 - accuracy: 0.0471 - mean_squared_error: 0.6568 - val_loss: 0.5908 - val_accuracy: 0.0662 - val_mean_squared_error: 0.5908
Epoch 2/100
44/44 [==============================] - 1s 24ms/step - loss: 0.5143 - accuracy: 0.0657 - mean_squared_error: 0.5143 - val_loss: 0.4574 - val_accuracy: 0.0658 - val_mean_squared_error: 0.4574
Epoch 3/100
44/44 [==============================] - 1s 24ms/step - loss: 0.3944 - accuracy: 0.0656 - mean_squared_error: 0.3944 - val_loss: 0.3674 - val_accuracy: 0.0658 - val_mean_squared_error: 0.3674
Epoch 4/100
44/44 [==============================] - 1s 24ms/step - loss: 0.3243 - accuracy: 0.0656 - mean_squared_error: 0.3243 - val_loss: 0.3147 - val_accuracy: 0.0658 - val_mean_squared_error: 0.3147
Epoch 5/100
44/44 [==============================] - 1s 25ms/step - loss: 0.2800 - accuracy: 0.0656 - mean_squared_error: 0.2800 - val_loss: 0.2782 - val_accuracy: 0.0658 - val_mean_sq

In [39]:
from sklearn.metrics import mean_squared_error
loss, accuracy,mean_squared_error = model_GRU2.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % accuracy)
print('Error Rate: %f' % (1 - accuracy))
print('Mean Squared Error: %f' % mean_squared_error)

Accuracy: 0.693400
Error Rate: 0.306600
Mean Squared Error: 0.137744
